# Lab 9: Linear Algebra Wrap Up

In this notebook, we'll look at a few remaining topics from our discussion of Linear Algebra including matrix diagonalization, Singular Value Decomposition (SVD), and Rank Reduction using SVD. 

In [ ]:
#import a few packages we'll need to get started
import numpy as np
import matplotlib.pyplot as plt

### Activity 1: Matrix Diagonalization

Recall from class that matrix diagonalization is accomplished using the relation:

$$
A = P D P^{-1}
$$

where D is a diagonal matrix containing the eigenvalues of A along its diagonal and P is the matrix whose columns are the eigenvectors of A. Note for this process to work, the matrix P must be invertable, and this only is allowed for square matricies. 

In order to diagonalize a matrix, we must:
* Compute eigenvalues and eigenvectors of A
* Form diagonal matrix D from eigenvalues
* Form the matrix P from the eigenvectors
* Calculate P inverse
* Finally check that the relation $A = PDP^{-1}$ is satisfied

With the above instructions in mind, take a moment to review the code below. It goes through the process of constructing the matrix D and P for a given matrix A

In [ ]:
def m_diag(A):
    n = A.shape[0]
    
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(A)
    
    # Form the diagonal matrix D with eigenvalues on the diagonal
    D = np.zeros((n, n))
    for i in range(n):
        D[i, i] = eigenvalues[i]
    
    # The matrix P has eigenvectors as its columns
    P = eigenvectors.copy()
    
    # Compute det(P), if small the matrix may not be invertable
    if np.abs(np.linalg.det(P))<1E-3:
        print("Warning: Matrix P is ill conditioned. Matrix A may not be diagonalizable.")
        P_inverse = None
    
    return D, P

Define an example matrix A, then use our m_diag() function on it. Print the results. 

In [ ]:
# Example Matrix A
A = np.array([[4, 2, 3],
              [1, 5, 2],
              [2, 2, 7]])


# get D and P from m_diag()
D, P = m_diag(np.float32(A)) #go ahead and say the matrix entries are float to avoid errors

#print results
print("\nDiagonal matrix D:")
print(D)

print("\nEigenvector matrix P:")
print(P)

Find and print P-inverse

In [ ]:
# Calculate inverse
P_inv = np.linalg.inv(P)

# print result
print("P^-1:")
print(P_inv)

Perform reconstruction of A using relation $\mathsf{PDP^{-1}$ = A}$

In [ ]:
print("Original matrix A:")
print(A)


#Calculate reconstruction - if you haven't seen this before @ is also multiplicaion symbol for matricies
A_reconstructed = P @ D @ P_inv

print("\nReconstruction A = P * D * P^(-1):")
print(A_reconstructed)


This result is close enough considering that we are likely to have some roundoff error, and applying modest rounding rules would return the original matrix.

Next, let's take a closer look at this process by constructing a matrix with properties that are our own design

In [ ]:
# Define eigenvalues to use
lambda1, lambda2, lambda3 = ..., ..., ...


# Define eigenvectors to use
v1 = np.array([ ... , ... , ...])
v2= np.array([ ... , ... , ...])
v3 = np.array([... , ... , ...])


Construct the matrix D from eigenvalues

In [ ]:
D = np.diag([lambda1,lambda2,lambda3])

print("\nEigenvector matrix D:")
print(D)

Construct the matrix P from the eigenvectors

In [ ]:
#construct the matrix P
P=np.column_stack((v1, v2, v3))

print("\nEigenvector matrix P:")
print(P)

For this process to work, P must be invertable. Try calculating the inverse. If this fails, then you might have to go back and adjust values in your eigenvectors.

In [ ]:
# Calculate inverse
P_inv = np.linalg.inv(P)

# print result
print("P^-1:")
print(P_inv)

Find the resulting matrix A

In [ ]:
#Calculate resulting matrix A
A_new = P @ D @ P_inv

print("\n Newly Constructed A:")
print(A_new)

Now verify that we get our original eigenvalues 

In [ ]:
#get eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(A_new)

print("Resulting Eigenvalues: ")
print(eigenvalues)

And now the resulting eigenvectors

In [ ]:
v1_result = eigenvectors[:,0] 
v2_result = eigenvectors[:,1]  
v3_result = eigenvectors[:,2]  

print("\nResulting Eigenvectors: ")
print("v1:", v1_result)
print("v2:", v2_result)
print("v3:", v3_result)

(optional) If you are not convinced v1, v2, v3 returned as exected, take a moment to work out the nomalization to verify the result

In [ ]:
#get normalization
vectornorm = np.linalg.norm(v2)

# re-scale the vector
scaled_vector = vectornorm*v1_result #note: you might have to switch which vector is referenced because they may not come back in the same order!

#print result
print(scaled_vector)

### Activity 2: Singular Value Decomposition

Recall from class that Singular Value Decomposition (SVD) is another type of matrix factorization technique that expresses a matrix as te producet of 3 matricies in the form:
$$
A = U \Sigma V^T
$$

Notably this is different from the last procedure because we are not using eigenvalues and eigenvectors of A, and also this techinque works on non-square matricies!

We'll reuse our earlier matrix A to start this activity.

In [ ]:
print("Matrix A:")
print(A)

Perform SVD and print the results

In [ ]:
# get SVD
U, sigma, VT = np.linalg.svd(A)

# note sigma are just the singular values, so we have to 'rebuild' it to the Sigma Matrix
Sigma = np.zeros(A.shape)
Sigma[:len(sigma), :len(sigma)] = np.diag(sigma)

print(f" Matrix U:")
print(U)
print(f"\nMatrix Sigma:")
print(Sigma)
print(f"\nMatrix V^T:")
print(VT)


Verify that we can use these results to reconstruct A

In [ ]:
#Calculate reconstruction
A_reconstructed = U @ Sigma @ VT

print("\nReconstruction A = U * \Sigma * V^T:")
print(A_reconstructed)

### Activity 3: SVD and Reducing Rank

In this activity, we will take a look at how SVD allows for dimensionality reduction as discussed in class. Specifically we will look at application to file compression 

In [ ]:
# import libraries needed
from skimage import data, color

We need to load in data for this activity. We will use an image of astronaut Eileen Collins that is included in the skimage package. Create a plot of the image to see what we are working with

In [ ]:
# Load a sample grayscale image
image = color.rgb2gray(data.astronaut())

# Print the shape of data
print('Shape of image data:')
print(np.shape(image))

# take a quick look at the image
plt.figure()
plt.imshow(image,cmap='gray')
plt.show()


While this is an image, remember that it is representable as a matrix. Run the cell below to print the first 10x10 entries, but remember the 'full' image is represented by a 512x512 matrix.

In [ ]:
#set how much of image matrix to show
n = 10

# print slice of image
print('Slice of small part of image matrix: ')
print(image[:n,:n])



Peform SVD on the image

In [ ]:
# Perform SVD
U, S, VT = np.linalg.svd(image)

Let's now look at reducing the deminsionality of data kept for reconstructing the image. Use the parameter K below to only keep the top 51 singular values (essentially 10%) of the original matrix data. 

In [ ]:
# Keep only the top 'k' singular values
k = ...

# use the specified k above to slice the SVD matricies appropriately
Sigma_k = np.diag(S[:k])
U_k = U[:, :k]
VT_k = VT[:k, :]

Perform the image reconstruction

In [ ]:
# Reconstruct the image using only the top k components
compressed_image = U_k @ Sigma_k @ VT_k

Plot the reconstructed image side by side with the original. How well has this process worked, even with keeping an small amount of the original data?

In [ ]:
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(compressed_image, cmap='gray')
plt.title(f'Compressed Image (k={k})')
plt.axis('off')

plt.tight_layout()
plt.show()

print(f"This uses only {k/len(S)*100:.2f}% of the singular values of the original image matrix!")

Try this out again using different amounts of data retained in the reconstruction by adjusting the value of 'k'. Try to answer the following questions
 - What is the minimum value needed to at least retain some of the original image that is recognisable? 
 - Can you 'by eye' tell any difference when 80, 85, 90, 95, ...% of the data is kept? 